In [1]:
using Pkg;
Pkg.add(url="https://github.com/FerreolS/OptimPackNextGen.jl",rev="verbose")


    Updating git-repo `https://github.com/FerreolS/OptimPackNextGen.jl`
    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`


In [2]:
include("../src/SphereIFSCalib.jl")
using TwoDimensional, Zygote,StatsBase, Plots, OptimPackNextGen
plotly()

Plots.PlotlyBackend()

In [3]:
# wavelengths
const λ1 = 987.72e-9# laser 1 
const λ2 = 1123.71e-9# laser 2 
const λ3 = 1309.37e-9# laser 3
const λ4 = 1545.10e-9  # laser 4  
λlaser = [λ1,λ2,λ3]
λ0 = mean(λlaser);# reference

In [4]:
# model of the LensletModel
bbox = BoundingBox(xmin=1, ymin=1, xmax=10, ymax=50);
a0= rand(Float64,length(λlaser));
fwhm0= rand(Float64,length(λlaser))*5;
laser =  LaserModel(λlaser,a0,fwhm0)
lmod = LensletModel(λ0,laser.nλ-1,bbox);



In [5]:

Cinit = zeros(2,laser.nλ)
Cinit[1:2,1:2] = [[ 6.2 10]; [20 100]]
UpdateDispModel(lmod.dmodel,Cinit)
limage = LensletLaserImage(lmod,laser)
heatmap(limage)


<!DOCTYPE html>
 
 
 Plots.jl

In [6]:
function  likelihood(a::Array{Float64,1},fwhm::Array{Float64,1},C::Array{Float64,2}) 
    UpdateDispModel(lmod.dmodel, C);
    UpdateLaserModel(laser,a,fwhm);
    return sum((data .- LensletLaserImage(lmod,laser)).^2)
end

data= limage ;
cost = likelihood(a0,fwhm0,Cinit)


0.0

In [7]:
@time ∇cost = gradient(likelihood,a0,fwhm0,Cinit)


 25.431170 seconds (59.66 M allocations: 3.407 GiB, 5.07% gc time, 99.93% compilation time)


([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0 0.0 0.0; 0.0 0.0 0.0])

In [8]:

@show  cost = likelihood(a0,fwhm0.+ 2.,Cinit)
@time  ∇cost = gradient(likelihood,a0,fwhm0 .+ 2.,Cinit)


cost = likelihood(a0, fwhm0 .+ 2.0, Cinit) = 6.969943420671099
  0.001166 seconds (7.79 k allocations: 414.078 KiB)


([10.800159556878793, 1.1169617839014108, 8.11704186750405], [3.2713934709045405, 0.03280038478100086, 3.0365278017468813], [-0.24870919392470384 -0.01728699707439444 -0.004678939467916648; 0.04954835745741402 -0.007363525246481786 0.00108723250768381])

In [9]:

function  likelihood(x::Vector{Float64}) 
    (a,fwhm,c) = (x[1:(laser.nλ)],x[(laser.nλ+1):(2*laser.nλ)],reshape(x[(2*laser.nλ+1):(4*laser.nλ)],2,:));
    UpdateDispModel(lmod.dmodel, c);
    UpdateLaserModel(laser,a,fwhm);
    return sum((data .- LensletLaserImage(lmod,laser)).^2)
end


likelihood (generic function with 2 methods)

In [10]:
x0 = vcat([a0[:],fwhm0[:],Cinit[:] ]...)
cost = likelihood(x0)
@time ∇cost = gradient(likelihood,x0)

@time  ∇cost = gradient(likelihood,x0)


  0.407699 seconds (1.23 M allocations: 71.578 MiB, 5.60% gc time, 99.56% compilation time)
  0.001141 seconds (7.91 k allocations: 423.750 KiB)


([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],)

In [11]:

xtrue = vcat([a0[:],fwhm0[:],Cinit[:]]...)
xinit = xtrue  .+ 1 .* rand(Float64,laser.nλ*4)
@time  ∇cost = gradient(likelihood,xinit )

@time xopt = vmlmb(likelihood, xinit; verb=10)

  0.001235 seconds (7.91 k allocations: 423.750 KiB)
# ITER   EVAL   REJECTS          F(X)           ||G(X)||    STEP
#-----------------------------------------------------------------
     0      1      0    7.1056010065623996E+00  1.15E+01  0.00E+00
    10     11      0    3.1104981493576246E-02  1.21E-01  1.00E+00
    20     21      0    2.5020515783477021E-03  4.29E-02  1.00E+00
    30     31      0    1.3901019249199390E-04  1.09E-02  1.00E+00
    40     42      0    4.0859656120289166E-05  4.09E-03  1.00E+00
    50     52      0    2.4167039356582776E-05  1.47E-03  1.00E+00
    60     63      0    9.0275081638031478E-06  1.38E-03  1.00E+00
    70     74      0    5.4043342076757309E-06  3.09E-04  1.00E+00
    80     85      0    4.6982168321339687E-06  2.58E-04  1.00E+00
    90     96      0    4.1340806473300199E-06  8.15E-05  1.00E+00
   100    107      0    4.1188857294429371E-06  2.57E-05  1.00E+00
   110    117      0    4.1139282209677309E-06  5.29E-05  1.00E+00
   120    1

12-element Vector{Float64}:
   0.9385244205924756
   0.09082458977175056
   0.9489454009463055
   2.33443870533708
   2.577787310075566
   1.142615140431624
   6.197615637964083
  19.998730526255066
  10.001750657016553
 100.00093230376315
   0.12087233937215228
   0.06424802657407089

In [12]:
(aopt,fwhmopt,copt) = (xopt[1:(laser.nλ)],xopt[(laser.nλ+1):(2*laser.nλ)],reshape(xopt[(2*laser.nλ+1):(4*laser.nλ)],2,:));
(ainit,fwhminit,cinit) = (xinit[1:(laser.nλ)],xinit[(laser.nλ+1):(2*laser.nλ)],reshape(xinit[(2*laser.nλ+1):(4*laser.nλ)],2,:));
@show(ainit,aopt,a0);

ainit = [1.6778825145745562, 0.9646573708633237, 1.6463554426157878]
aopt = [0.9385244205924756, 0.09082458977175056, 0.9489454009463055]
a0 = [0.9385263852370316, 0.0908231410651481, 0.9489611666378173]
